In [ ]:
!pip install lion_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from lion_pytorch import Lion

In [ ]:
model_id = "unsloth/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

### tokenizer Size
print(f"vocab size: {len(tokenizer.get_vocab())}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab size: 256000


In [ ]:
### Encoding and Decoding of text

text = "Here we go guys"
print(f"Original text: {text}")

## Tokenize the text
tokens = tokenizer.encode(text, return_tensors="pt")
print(f"Tokens: {tokens}")

## Decode the tokens
decoded_text = tokenizer.decode(tokens[0])
print(f"Decoded text: {decoded_text}")

Original text: Here we go guys
Tokens: tensor([[   2, 4858,  783,  871, 6494]])
Decoded text: <bos>Here we go guys


In [ ]:
### Load the model
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

In [ ]:
# Basic question-answer template
template_without_answer = "<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model\n"
template_with_answer = template_without_answer + "{answer}<end_of_turn>\n"

# Let's try to put something into the template to see how it looks
print(template_with_answer.format(question="What is your name?", answer="My name is Gemma!"))

<start_of_turn>user
What is your name?<end_of_turn>
<start_of_turn>model
My name is Gemma!<end_of_turn>



In [ ]:
### Putting it all together to prompt the model and generate a response

## 1. Construct a prompt a chat template form

prompt = template_without_answer.format(question="What is your name?")
print(prompt)

## 2. Tokenize the prompt
tokens = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

## 3. Feed through the model to predict the next token probabilities
with torch.no_grad():
    output = model(tokens)
    probs = F.softmax(output.logits, dim=-1)

# 4. Get the next token, according to the maximum probability
next_token = torch.argmax(probs[0, -1, :]).item()

# 5. Decode the next token
next_token_text = tokenizer.decode([next_token])

# Print results
print(f"Prompt: {prompt}")
print(f"Predicted next token: {next_token_text}")

<start_of_turn>user
What is your name?<end_of_turn>
<start_of_turn>model

Prompt: <start_of_turn>user
What is your name?<end_of_turn>
<start_of_turn>model

Predicted next token: My


In [ ]:
prompt = template_without_answer.format(question="What does MIT stand for?")
tokens = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
output = model.generate(tokens, max_new_tokens=20)
print(tokenizer.decode(output[0]))

<bos><start_of_turn>user
What does MIT stand for?<end_of_turn>
<start_of_turn>model
MIT stands for **Massachusetts Institute of Technology**.<end_of_turn>


In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader, random_split



In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

# Step 1: Load the parquet file
file_path = "/content/drive/MyDrive/vamps/train-00000-of-00001.parquet"
df = pd.read_parquet(file_path)

# Step 2: Map labels to strings
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Step 3: Transform the DataFrame into desired format
class FinancialPhraseBankDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe['train'] # Access the 'train' column
        self.data = self.data.apply(pd.Series) # Convert dictionaries to columns
        self.data["instruction"] = (
            "What is the sentiment of the following financial sentence?\n\n" + self.data["sentence"]
        )
        self.data["response"] = self.data["label"].map(label_map)
        self.data["response_style"] = "financial sentiment analysis"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        return {
            "instruction": row["instruction"],
            "response": row["response"],
            "response_style": row["response_style"]
        }

# Step 4: Create Dataset and DataLoaders
full_dataset = FinancialPhraseBankDataset(df)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
sample = train_loader.dataset[44]


In [ ]:
sample

{'instruction': "What is the sentiment of the following financial sentence?\n\nAmid the swiftly evolving energy landscape, on 15 November 2010, Finnish energy company Fortum Oyj HEL : FUM1V unveiled its commencement of negotiations with partners to establish a new operating model for energy supply in the Turku region of western Finland. This development signals the company's proactive stance towards adapting to changing market dynamics and enhancing its operational strategies.",
 'response': 'neutral',
 'response_style': 'financial sentiment analysis'}

In [ ]:
question = sample['instruction']
answer = sample['response']
answer_style = sample['response_style']

print(f"Question: {question}\n\n" +
      f"Original Answer: {answer}\n\n" +
      f"Answer Style: {answer_style}")

Question: What is the sentiment of the following financial sentence?

Amid the swiftly evolving energy landscape, on 15 November 2010, Finnish energy company Fortum Oyj HEL : FUM1V unveiled its commencement of negotiations with partners to establish a new operating model for energy supply in the Turku region of western Finland. This development signals the company's proactive stance towards adapting to changing market dynamics and enhancing its operational strategies.

Original Answer: neutral

Answer Style: financial sentiment analysis


In [ ]:
def chat(question, max_new_tokens=32, temperature=0.7, only_answer=False):
    ## 1. Construct the prompt using template
    prompt = template_without_answer.format(question=question)

    ## 2. Tokenize the text
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    ## 3. Feed though the model to predict next token probability
    with torch.no_grad():
      outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)

    ## 4. Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    ## 5. Extract the answer
    #answer = result.replace(prompt, "").replace("\n", "")

    return result

In [ ]:
### let's try chat with model

ans = chat("what is the capital of india", only_answer=True)
print(ans)

In [ ]:
### LORA is to finetune the model

def apply_lora(model):
  ## Define lora config
  lora_config = LoraConfig(
      r=8,
      task_type="CAUSAL_LM",
      target_modules=[
            "q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"
        ],
  )

  ## Apply lora in the model
  lora_model = get_peft_model(model, lora_config)
  return lora_model

In [ ]:
fine_model = apply_lora(model)

# Print the number of trainable parameters after applying LoRA
trainable_params = sum(p.numel() for p in fine_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in fine_model.parameters())
print(f"number of trainable parameters: {trainable_params}")
print(f"total parameters: {total_params}")
print(f"percentage of trainable parameters: {trainable_params / total_params * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


number of trainable parameters: 10383360
total parameters: 2624725248
percentage of trainable parameters: 0.40%


In [ ]:
def forward_and_compute_loss(model, tokens, mask, context_length=512):
    # Truncate to context length
    tokens = tokens[:, :context_length]
    mask = mask[:context_length]

    # Construct the input, output, and mask
    x = tokens[:, :-1]
    y = tokens[:, 1:]
    mask = mask[1:]

    # Forward pass to compute logits
    logits = model(x).logits

    # Compute loss
    loss = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        y.view(-1),
        reduction="none"
    )

    # Mask out the loss for non-answer tokens
    loss = loss[mask.view(-1)].mean()

    return loss

In [ ]:
### Training loops

def train(model, dataloader, tokenizer, max_step=5, context_length=512, learning_rate=1e-4):
  losses = []

  ### Apply lora in the model
  model = apply_lora(model)

  optimizer = Lion(model.parameters(), lr=learning_rate)

  ## Training loop
  for step, batch in enumerate(dataloader):
    question = batch["instruction"][0]
    answer = batch["response"][0]

    ## format the question and answer into the template
    text = template_with_answer.format(question=question, answer=answer)

    ## Tokenize the question and answer
    ids = tokenizer(text, return_tensors="pt", return_offsets_mapping=True).to(model.device)

    # Extract input_ids and create the mask
    input_ids = ids["input_ids"]
    offset_mapping = ids["offset_mapping"][0]
    answer_start = text.index(answer)
    mask = (offset_mapping[:, 0] >= answer_start).squeeze()


    ## Feed the tokens to the model snd compute the loss
    loss = forward_and_compute_loss(model, input_ids, mask)

    ## Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss.item())

    # monitor progress
    if step % 1 == 0:
      #print(chat("What is the capital of India?", only_answer=True))
      print(f"step {step} loss: {torch.mean(torch.tensor(losses)).item()}")
      losses = []

    if step > 0 and step % max_step == 0:
        break

  return model

In [ ]:
model = train(model, train_loader, tokenizer)

step 0 loss: 14.91193675994873
step 1 loss: 13.927783012390137
step 2 loss: 8.629654884338379
step 3 loss: 5.745876789093018
step 4 loss: 5.233701229095459
step 5 loss: 6.369597911834717


In [ ]:
def chat(prompt, only_answer=True, max_new_tokens=50):
    # Format input as a chat or prompt template
    input_text = f"### Financial News:\n{prompt}\n### Sentiment:"

    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False  # deterministic output
        )

    # Decode generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the sentiment
    if only_answer:
        if "### Sentiment:" in generated_text:
            return generated_text.split("### Sentiment:")[-1].strip()
        else:
            return generated_text.strip()
    else:
        return generated_text


In [ ]:
ans = chat("Shares of the company surged to a high of Rs 2,994.5 per share on the NSE, rising 7 percent. The stock has been gaining for the last two days. What is the sentiment ?", only_answer=True)

In [ ]:
ans

'The sentiment is **positive**. \n\n**Explanation:**\n\n* **Surge in share price:** A significant increase in share price indicates positive investor sentiment. \n* **High price:** The high price of Rs 2,994.'

In [ ]:
# prompt: Write a code save the trained model and it tokenizer in my local systm.

from google.colab import files

# Define the directory where you want to save the model and tokenizer
save_directory = "./finetuned_gemma"

# Save the fine-tuned model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

# Download the saved directory as a zip file
!zip -r /content/finetuned_gemma.zip /content/finetuned_gemma
files.download('/content/finetuned_gemma.zip')

Model and tokenizer saved to ./finetuned_gemma
  adding: content/finetuned_gemma/ (stored 0%)
  adding: content/finetuned_gemma/special_tokens_map.json (deflated 76%)
  adding: content/finetuned_gemma/adapter_model.safetensors (deflated 48%)
  adding: content/finetuned_gemma/tokenizer.model (deflated 51%)
  adding: content/finetuned_gemma/chat_template.jinja (deflated 52%)
  adding: content/finetuned_gemma/tokenizer_config.json (deflated 96%)
  adding: content/finetuned_gemma/adapter_config.json (deflated 58%)
  adding: content/finetuned_gemma/tokenizer.json (deflated 84%)
  adding: content/finetuned_gemma/README.md (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: write a code to load this model in notebook and inference with it.

# Assuming the fine-tuned model and tokenizer were saved to './finetuned_gemma' and downloaded as 'finetuned_gemma.zip' previously
# If you are running this in a new notebook session, you need to upload the zip file first.
# Then unzip it to the desired directory (e.g., './finetuned_gemma').

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from google.colab import files
import zipfile

# If you need to upload the zip file:
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# If you uploaded 'finetuned_gemma.zip', you need to unzip it
# unzip_path = './finetuned_gemma'
# os.makedirs(unzip_path, exist_ok=True)
# with zipfile.ZipFile('finetuned_gemma.zip', 'r') as zip_ref:
#     zip_ref.extractall(unzip_path)
# print(f"Unzipped model to {unzip_path}")

# Define the directory where the fine-tuned model and tokenizer are located
# Update this path if you unzipped to a different location
model_directory = "./finetuned_gemma"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Load the base model first (as PEFT models require the base model)
# Use the original model_id or specify the path to the base model if available locally
base_model_id = "unsloth/gemma-2-2b-it"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16) # Use float16 for efficiency

# Load the PEFT model (LoRA adapters) on top of the base model
model = PeftModel.from_pretrained(base_model, model_directory)

# Merge the LoRA weights into the base model for easier inference (optional, but good practice)
# This creates a single model with merged weights
model = model.merge_and_unload()

# Set the model to evaluation mode
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded successfully and set to evaluation mode.")
print(f"Model is on device: {device}")

# Define the chat function for inference
def chat_inference(prompt, max_new_tokens=50, temperature=0.7, only_answer=True):
    # Format input as a chat or prompt template - match the training format
    input_text = f"### Financial News:\n{prompt}\n### Sentiment:"

    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True if temperature > 0 else False, # Use sampling if temperature > 0
            temperature=temperature if temperature > 0 else 1.0,
            top_k=50, # Example sampling parameter
            top_p=0.95 # Example sampling parameter
        )

    # Decode generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the sentiment
    if only_answer:
        if "### Sentiment:" in generated_text:
            # Find the part after "### Sentiment:" and strip whitespace
            sentiment_part = generated_text.split("### Sentiment:", 1)[-1].strip()
            # Take only the first word as the sentiment (assuming sentiment is a single word)
            return sentiment_part.split()[0] if sentiment_part else ""
        else:
            # If the marker is not found, return the full generated text stripped
            return generated_text.strip()
    else:
        return generated_text

# Example inference
test_sentence = "Company announced strong earnings, exceeding analyst expectations."
sentiment = chat_inference(test_sentence, only_answer=True, temperature=0.1) # Use low temperature for more deterministic output

print(f"\nSentence: {test_sentence}")
print(f"Predicted Sentiment: {sentiment}")

# Another example
test_sentence_2 = "The company's stock price dropped significantly after the product recall."
sentiment_2 = chat_inference(test_sentence_2, only_answer=True, temperature=0.1)

print(f"\nSentence: {test_sentence_2}")
print(f"Predicted Sentiment: {sentiment_2}")

# Example with a different sentiment
test_sentence_3 = "The market remained calm today with little change in stock prices."
sentiment_3 = chat_inference(test_sentence_3, only_answer=True, temperature=0.1)

print(f"\nSentence: {test_sentence_3}")
print(f"Predicted Sentiment: {sentiment_3}")


In [ ]:
# prompt: how to use this downloaded model for the sentimental analyis in a flask application

from flask import Flask, request, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

app = Flask(__name__)

# Define the directory where you saved the model and tokenizer
SAVE_DIRECTORY = "./finetuned_gemma" # Adjust this path to where you extract the zip file

# Load the fine-tuned model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(SAVE_DIRECTORY)
    # Ensure model is loaded onto the appropriate device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(SAVE_DIRECTORY).to(device)
    print(f"Model and tokenizer loaded successfully from {SAVE_DIRECTORY}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    tokenizer = None
    model = None

def analyze_sentiment(text):
    """
    Analyzes the sentiment of the given text using the loaded model.
    """
    if model is None or tokenizer is None:
        return "Error: Model or tokenizer not loaded."

    # Format input as a chat or prompt template
    input_text = f"### Financial News:\n{text}\n### Sentiment:"

    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50, # You can adjust this
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False  # deterministic output
        )

    # Decode generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the sentiment
    if "### Sentiment:" in generated_text:
        return generated_text.split("### Sentiment:")[-1].strip()
    else:
        # Fallback if the expected format is not found
        return generated_text.strip()

@app.route('/analyze_sentiment', methods=['POST'])
def sentiment_api():
    """
    API endpoint to receive text and return its sentiment.
    """
    if not request.json or 'text' not in request.json:
        return jsonify({"error": "Please provide 'text' in the request body"}), 400

    text_to_analyze = request.json['text']
    sentiment = analyze_sentiment(text_to_analyze)

    return jsonify({"sentiment": sentiment})

if __name__ == '__main__':
    # To run the Flask app locally
    app.run(debug=True)


In [ ]:
# prompt: create a code using gardio liberary, write in the format of app.py for financial analysis that i can directly use in the hugging space

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

# Define the directory where you saved the model and tokenizer
# This should be the directory you created when saving the model in Colab
SAVE_DIRECTORY = "./finetuned_gemma" # Adjust this path if you saved it elsewhere

# Load the fine-tuned model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(SAVE_DIRECTORY)
    # Ensure model is loaded onto the appropriate device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(SAVE_DIRECTORY).to(device)
    model.eval() # Set model to evaluation mode
    print(f"Model and tokenizer loaded successfully from {SAVE_DIRECTORY}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    tokenizer = None
    model = None
    device = "cpu" # Set device to cpu if loading fails

def analyze_sentiment_gradio(text):
    """
    Analyzes the sentiment of the given text using the loaded model
    for Gradio interface.
    """
    if model is None or tokenizer is None:
        return "Error: Model or tokenizer not loaded. Please ensure the model directory exists and contains the necessary files."

    if not text:
        return "Please enter some financial text to analyze."

    # Format input as a chat or prompt template
    input_text = f"### Financial News:\n{text}\n### Sentiment:"

    try:
        # Tokenize
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50, # You can adjust this
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False  # deterministic output
            )

        # Decode generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the sentiment
        if "### Sentiment:" in generated_text:
            return generated_text.split("### Sentiment:")[-1].strip()
        else:
            # Fallback if the expected format is not found
            # You might want to refine this based on your model's output
            return generated_text.strip()

    except Exception as e:
        return f"An error occurred during analysis: {e}"

# Create the Gradio interface
if model is not None and tokenizer is not None:
    gr.Interface(
        fn=analyze_sentiment_gradio,
        inputs=gr.Textbox(lines=5, label="Enter Financial News Text"),
        outputs=gr.Textbox(label="Sentiment"),
        title="Financial News Sentiment Analysis",
        description="Analyze the sentiment of financial news headlines and sentences using a fine-tuned language model."
    ).launch()
else:
    print("Gradio interface could not be launched because the model or tokenizer failed to load.")



In [ ]:
# prompt: create the requirements in the format of requirements.txt that will required to run the above file

transformers
datasets
torch
tqdm
matplotlib
pandas
peft
lion-pytorch
flask
gradio
numpy
